In [ ]:
df_trip = pd.read_csv("liste musées 2.csv", index_col=0)
df_url = pd.read_csv("url musées tripadvisor.csv",index_col=0)
df_trip.rename(columns={"avis" : "reviews"}, inplace=True)
df_trip

In [101]:
df_trip.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4192 entries, 0 to 4191
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        4174 non-null   object
 1   rating       3818 non-null   object
 2   avis         3818 non-null   object
 3   labels       4170 non-null   object
 4   adress       3958 non-null   object
 5   coordinates  4192 non-null   object
dtypes: object(6)
memory usage: 229.2+ KB


# cleaning title

In [ ]:
print(len(df_trip[df_trip["title"].isna()]), "museums without a title")
df_trip[df_trip["title"].isna()]

In [124]:
#df_trip.loc[1125,["title","rating",'avis',"labels","adress"]] = ["Fondation du Doute", "bubble_40", "151 avis","Musées spécialisés","14 Rue de la Paix, 41000 Blois"]
#df_trip.loc[1509,"title"] = "Maison du Patrimoine Edouard Boeglin"
df_trip.loc[1594, "title"] = "Musee de la Vigne Et du Champagne"
df_trip.loc[1936,["title","rating",'avis',"labels","adress"]] = ["Musée de la Libération de Paris - Musée du Général Leclerc - Musée Jean Moulin","bubble_40", "99 avis", "Musées militaires, Musées d'histoire", "Place Denfert-Rochereau 4 Avenue Du Colonel Henri Rol-Tanguy, 75014 Paris France"]
df_trip.loc[2111,"title"] = "Bibliothèque Nationale de France - Richelieu Site"
df_trip.loc[3152, "title"] = "Musée de la Résistance de la Déportation et de la Libération du Département du Lot"
df_trip = df_trip[~df_trip["title"].isna()]

## cleaning rating

In [ ]:
df_trip["rating"] = df_trip["rating"].str.replace("bubble_","").astype("float32")/10
df_trip

In [ ]:
df_trip['avis'] = df_trip['avis'].str.replace(" avis","")
df_trip['avis'] = df_trip['avis'].str.replace("NaN","0") 
df_trip['avis'] = df_trip['avis'].str.replace(" ","")
df_trip['avis'] = df_trip['avis'].str.replace("\xa0","")
df_trip['avis'] = pd.to_numeric(df_trip['avis'])
df_trip

## labels

In [187]:
df_trip = df_trip[df_trip["labels"] != "Galeries d'art"]

## adress

In [177]:
df_trip['city'] = df_trip['adress'].str.extract("\d{5}(.*)F")
df_trip['city'] = df_trip['city'].str.replace(",","")
df_trip["postal_code"] = df_trip["adress"].str.extract("(\d{5})")
df_trip["adress"] = df_trip["adress"].str.replace(" France", "")

C:\Users\richa\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\richa\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\richa\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [ ]:
def clean_tup(x):
    x = x.lstrip("(")
    x = x.rstrip(")")
    x = x.split(",")
    return x
df_trip["coordinates"] = df_trip["coordinates"].apply(clean_tup)

temp = df_trip["coordinates"].apply(pd.Series)
temp.columns = ["latitude","longitude"]
df_trip = pd.concat([df_trip,temp],axis=1)
df_trip["latitude"] = df_trip["latitude"].str.replace("'","").replace('"',"").astype("float64")
df_trip["longitude"] = df_trip["longitude"].str.replace("'","").replace('"',"").astype("float64")
df_trip.drop("coordinates", axis=1,inplace=True)

## using reverse geocoding to find missing adress

In [298]:
import requests
import time
to_reverse_geo = df_trip[(df_trip["postal_code"].isna()) & (~df_trip["longitude"].isna())]
coo_list = to_reverse_geo['latitude'].astype(str) + "," + to_reverse_geo['longitude'].astype(str)
city_list = []
pcode_list = []
api_url = "http://dev.virtualearth.net/REST/v1/Locations/{}?o=json&key=AlTWqR9w6rm__Qib_WPX-9f3K9SEilwv5Sg4ZTJAbPpR8NsYil_04aHnFCnMr9EE"

for coordinates in coo_list:
    response = requests.get(api_url.format(coordinates))
    r = response.json()
    try: 
        city = r["resourceSets"][0]["resources"][0]["address"]["locality"]
    except Exception as e: 
        city = "NaN"
        print(e)
    try: 
        postal_code = r["resourceSets"][0]["resources"][0]["address"]["postalCode"]
    except Exception as e: 
        postal_code = "NaN"
        print(e)

    city_list.append(city)
    pcode_list.append(postal_code)
    print(postal_code)
    time.sleep(0.5)

df_trip.loc[to_reverse_geo.index, "city"] = city_list
df_trip.loc[to_reverse_geo.index, "postal_code"] = pcode_list

## adding departement name

In [ ]:
df_dep = pd.read_html("https://www.regions-et-departements.fr/departements-francais")[0]
df_dep = df_dep.replace("2B","20").replace("2A","20").rename(columns={"NUMÉROdu département" : "dep_num", "NOMdu département" : "dep_name"})
df_dep

In [338]:
df_trip["dep_code"] = df_trip["postal_code"].apply(lambda x : x[:2] if pd.notnull(x) else x)
df_trip=df_trip.merge(df_dep, left_on="dep_code", right_on="dep_num", how="left")

C:\Users\richa\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [340]:
df_trip = df_trip.drop_duplicates(subset=["title","city"])
df_trip.drop("dep_num",axis=1, inplace=True)

In [346]:
df_trip.to_csv("musées tripadvisor clean.csv")